In [2]:
# usage: $python3 CULane_Convert_LaneNet.py
# Will save converted CULANE into the same location that this script is run.

## Imports

In [3]:
from shutil import copy
from shutil import copyfile
import os
import sys
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from os import walk
import IPython
#IPython.embed() # to debug in notebook
import random
print("Successfully imported all")

Successfully imported all


## Constants and paths

In [4]:
cwd = os.getcwd() + "/"
print("CWD: ", cwd)

CWD:  /root/atlas_root/home/cjcramer/lane_lines/Scripts/


In [5]:
display_results = False if sys.argv[-1] == "--no-vis" else True

In [6]:
export_folder_name = cwd + "culane_dataset_lanenet/"
print("Export folder: ", export_folder_name)

export_img_dir = export_folder_name + "image/"
export_instance_dir = export_folder_name + "gt_image_instance/"
export_binary_dir = export_folder_name + "gt_image_binary/"

# CHANGE THIS TO THE LOCATION OF YOUR LANENET DATA FOLDER
base_export_dir = "/root/root_dit_atlas/home/cjcramer/lane_lines/LaneNet/data/culane_dataset_lanenet/"
print("base_export_dir: ", base_export_dir)

image_dir = base_export_dir + "image/"
instance_dir = base_export_dir + "gt_image_instance/"
binary_dir = base_export_dir + "gt_image_binary/"
validation_path = base_export_dir + "val.txt"
train_path = base_export_dir + "train.txt"

# CHANGE THIS TO THE LOCATION OF YOUR CULANE DATASET
data_base_dir = "/root/atlas_root/teams/team-1/Datasets/CULane_Dataset/"
print("CULane path: ", data_base_dir)

# Folders of annotations
annotation_dir = data_base_dir + "laneseg_label_w16/"

# Take all images from annotation directory, get corresponding actual image from CULane,
# convert the image into annotations with values 20, 70, 120, 170
# convert the image to all white annotations

Export folder:  /root/atlas_root/home/cjcramer/lane_lines/Scripts/culane_dataset_lanenet/
base_export_dir:  /root/root_dit_atlas/home/cjcramer/lane_lines/LaneNet/data/culane_dataset_lanenet/
CULane path:  /root/atlas_root/teams/team-1/Datasets/CULane_Dataset/


## Convert notebook to python file

In [7]:
#!jupyter nbconvert --to script CULane_Remake.ipynb
# Can use the command line argument instead:
#$jupyter nbconvert --to script CULane_Remake.ipynb

## Print pretty with colors

In [8]:
# Credit: https://stackoverflow.com/questions/287871/print-in-terminal-with-colors
class CMD_C:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'   # End formatting
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## Helper functions

In [9]:
def make_dir(dir_path):
    if (os.path.exists(dir_path)):
        print(dir_path, CMD_C.OKGREEN, " already exists in the current working direcrectory: ", CMD_C.ENDC, cwd, sep="")
    else:
        try:
            os.mkdir(dir_path)
        except OSError:
            print(FAIL, "Could not create destination folder: ", dir_path, ENDC)
            return False
        else:
            print("Sucessfully made destination folder: ", dir_path)
    return True

# Gather all file paths

In [10]:
# Credit: https://www.mkyong.com/python/python-how-to-list-all-files-in-a-directory/
annotation_paths = []
for r, d, files in os.walk(annotation_dir):
    for file in files:
        annotation_paths.append(os.path.join(r, file))

print("Number of annotations total: ", CMD_C.OKBLUE, len(annotation_paths), CMD_C.ENDC, sep="")

Number of annotations total: 98877


# Gather corresponding actual pictures 

In [12]:
paths = []   # CULane img, CULane annot, In LaneNet img, in LaneNet binary, in LaneNet inst, copy to img, copy to binary, copy to inst 
new_file_name_counter = 0      # Since CULane has many files with the same name in different directories, enumerate the pictures.
num_blank = 0

print("Gathering paths and determining which images are blank...")
percent = len(annotation_paths) // 100
print_at = percent

for i, annot_path in enumerate(annotation_paths):
    # Skip past all blank pictures
    img = cv2.imread(annot_path, cv2.IMREAD_COLOR)
    if np.sum(img) == 0:
        num_blank += 1
        continue
    
    tok = annot_path.split("/")
    filename = tok[-1]
    filename_no_ext = filename.split(".")[0]
    path = "/".join(tok[-3:-1])
    path_to_file = data_base_dir + path + "/" + filename_no_ext + ".jpg"
    # Check to see if that file actually exists, because not all annotated pictures have pitures?
    # Or I deleted some pictures?
    # Should /CULane_Dataset/driver_161_90frame/06031716_0888.MP4/ be empty?
    if os.path.isfile(path_to_file) == False:
        continue
    
    image_path_lanenet = image_dir + str(new_file_name_counter) + ".png"
    binary_path_lanenet = binary_dir + str(new_file_name_counter) + ".png"
    instance_path_lanenet = instance_dir + str(new_file_name_counter) + ".png"
   
    export_image_path_lanenet = export_img_dir + str(new_file_name_counter) + ".png"
    export_binary_path_lanenet = export_binary_dir + str(new_file_name_counter) + ".png"
    export_instance_path_lanenet = export_instance_dir + str(new_file_name_counter) + ".png"
    
    paths.append((path_to_file, annot_path, image_path_lanenet, binary_path_lanenet, instance_path_lanenet, export_image_path_lanenet, export_binary_path_lanenet, export_instance_path_lanenet))
    new_file_name_counter += 1
    
    if i >= print_at:
        print(str(print_at) + "/" + str(len(annotation_paths)), "so far, num blanks = ", str(num_blank))
        print_at += percent 
    
print("Number blank annotations: ", num_blank)
print("Example paths")
for i in range(0, 2):
    print(paths[i][0])
    print(paths[i][1])
    print(paths[i][2])
    print(paths[i][3])
    print(paths[i][4])
    print(paths[i][5])
    print(paths[i][6])
    print(paths[i][7], "\n")

Gathering paths and determining which images are blank...
294138139
334866706
367777396
379944055
352875537
331015859
320538433
360896441
351691228
329910768
315595105
326001380
330180042
340838747
333459879
322577296
265764455
325599716
322604588
333557286
346944505
337066350
365672782
358227444
362865556
355811340
369164712
364123426
311379937
312061090
359975190
357525342
359509009
361549869
350679117
354811861
337926337
360776503
339472829
292611602
336605462
348045121
360760856
293127609
392420611
327816863
301451654
297777571
324291677
359662487
334341522
338226335
296322386
329296747
299066399
298578783
300674918
341830884
305033372
286855024
363930545
358585248
297235611
305126176
376688418
267835334
298261590
392448228
301906975
280838059
309667348
304933240
253834881
290711229
298864220
368661211
291415209
297058178
297614680
264190203
268970563
278190745
306774722
296692059
297959739
296154392
352994545
298952577
315059671
299573306
264675382
106307031
312725276
283213731
29

109370637
80978720
106760610
48162883
66252031
91287576
94532309
120167477
82922178
85230052
82057090
95559653
68861329
91202011
67785706
59401040
85110223
75997530
88925432
77783737
40539937
86112256
67867082
63762280
85782130
119335538
70132438
96919887
90814494
50973975
84747690
84122821
78020077
72149772
85948368
73021562
86614991
93036948
60167785
71097670
91588481
118890741
39176966
56955154
83860083
85616254
89593729
72148053
119012648
58155377
29225237
48323929
47062743
110470257
81378264
79038113
33504696
46469695
45519802
117273827
55936211
54638425
46164137
35178922
30696985
37177789
107053321
52027627
46832141
59735021
49117462
45902226
35141017
86545447
45912989
33470341
36708244
46260845
77859118
46290374
37152521
46483262
45339699
42066574
105337465
50865644
30260090
85107062
33734361
45993872
75917363
45132184
41284224
28907180
68933710
126781581
42480548
167393795
988/98877 so far, num blanks =  0
43526904
49047604
98331824
39005696
38792143
45887023
42295772
45504745


261351273
279449905
298228347
341115462
301336683
313613746
300374440
235940563
346745060
371263515
362073427
328270032
353449984
289353402
257958969
370993619
276321549
251146202
253719980
352527195
359606275
395403122
350163381
255055223
105236051
330938587
325901565
302012682
199454641
297043606
242663556
330045808
293075842
103436945
99731380
105143088
99622569
62573509
80646967
146007867
118577392
98913975
120251819
82746648
50867544
70499871
105200058
103710907
114514102
87803151
91081159
118508861
123738066
48983771
115368422
80490680
74778044
107042185
101216047
135689516
97971760
116320755
90254805
93119671
114198346
104866049
49636397
64386660
104172841
97244445
121799393
69964431
129810720
78619651
62251442
78497765
74256100
96719968
125844172
110890814
94189857
57955170
166752961
76878942
116610695
82557969
74974451
106026144
62099480
80691119
61684092
106661831
116873877
348079667
348854330
349765638
346538784
348152545
298600666
349061216
349256122
349647957
350022417
348

KeyboardInterrupt: 

# Make destination folder to save all outputs to

In [ ]:
folders_to_create = [export_folder_name, export_folder_name + "image/", export_folder_name + "gt_image_instance/", export_folder_name + "gt_image_binary/"]
for folder in folders_to_create:
    if make_dir(folder) == False:
        print(CMD_C.FAIL, "COULD NOT CREATE THE FOLDER: ", folder, CMD_C.ENDC, sep="")

# Save train.txt and val.txt

In [ ]:
# 0.9 to 0.1 train val split
with open(export_folder_name + "train.txt", "w") as train_file:
    with open(export_folder_name + "val.txt", "w") as val_file:
        for _, _, img_path, bin_path, inst_path, _, _, _ in paths:
            str_to_write = img_path + " " + bin_path + " " + inst_path + "\n"
            if random.random() < 0.9:
                train_file.write(str_to_write)
            else:
                val_file.write(str_to_write)
print("Wrote train.txt and val.txt")

## Main loop

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
#r_ind = 43456  # A random curvy section
file_no = 0
for CU_img_path, CU_annot_path, _, _, _, export_img_path, export_bin_path, export_inst_path in paths: 
    #filename_no_ext = CU_img_path.split("/")[-1].split(".")[0]
    file_no += 1
    print("Processing image ", CMD_C.OKGREEN, file_no, CMD_C.ENDC, " / ", len(paths), " : ", sep="", end="")
    print(CMD_C.OKGREEN,  "{0:.1f}".format(100 * file_no / len(paths)), "% | ", CMD_C.ENDC, sep="", end="")
    
    # Copy the real image to the image/ folder
    try:
        # Check if image already exists, and we will overwrite it
        if os.path.isfile(export_img_path) == True:
            print(CMD_C.FAIL, export_img_path, " already exists, DO NOT WANT TO OVERWRITE IT SO SKIPPING", CMD_C.ENDC)
            continue
        img = cv2.imread(CU_img_path, cv2.IMREAD_COLOR)
        save_success = cv2.imwrite(export_img_path, img)
        if save_success == False:
            print(CMD_C.FAIL, "Could not copy file: ", CU_img_path, CMD_C.ENDC, " to ", CMD_C.FAIL, export_img_path, CMD_C.ENDC, sep="", end="")
    except:
        print(CMD_C.FAIL, "Could not copy file: ", CU_img_path, CMD_C.ENDC, " to ", CMD_C.FAIL, export_img_path, CMD_C.ENDC, sep="", end="")
    else:
        print(" copied img | ", sep="", end="")
    
    #IPython.embed() # to debug in notebook
    #print("Exiting")
    #break
    
    # Read in the annotation image from CULane
    try:
        img = cv2.imread(CU_annot_path, cv2.IMREAD_GRAYSCALE)
        # Convert it to all white
        ret, white = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)
        # Convert the 1,2,3,4 annotated pixels to 20 70 120 170 respectively
        change_scale = img.copy()
        change_to = {1:20, 2:70, 3:120, 4:170}
        #print(change_scale)
        for old, new in change_to.items():
            change_scale[change_scale == old] = new
            #change_scale[np.where((change_scale == [1]).all(axis = 0))] = [20]
        #print(change_scale)
        # Save those two images
    except Exception as ex:
        print("Error", ex)
    else:
        pass
    
    # Save white
    save_success = cv2.imwrite(export_bin_path, white)
    if save_success:
        print("binary saved | ", sep="", end = "")
    else:
        print(CMD_C.FAIL, "FAILED TO SAVE IMAGE", CMD_C.ENDC, export_bin_path)
    
    # Save change_scale
    save_success = cv2.imwrite(export_inst_path, change_scale)
    if save_success:
        print("annotation saved | ", sep="", end = "")
    else:
        print(CMD_C.FAIL, "FAILED TO SAVE IMAGE", CMD_C.ENDC, export_inst_path)
    
    # Show result for first frame if display results
    if display_results == True:
        f, axs = plt.subplots(1, 3)
        for _, ax in np.ndenumerate(axs):   # Remove axes
            ax.axis('off')
        axs[0].imshow(img)
        axs[1].imshow(white)
        axs[2].imshow(change_scale)
        axs[0].set_title("Original (false color)")
        axs[1].set_title("Annotations all white")
        axs[2].set_title("Change Scale")
        plt.tight_layout()
        plt.show()
        break
    print()

In [ ]:
print("All done!")